# Ex 11: The stability of spin motion for an irregular asteroid

To study the stability of spin motion for an irregular shape we need a mass model to represent it. In the case of this asteroid we are going to use the following:

|z = 1||z = 0||z = -1||
--|--|--|--|--|--
x|y|x|y|x|y
-1|4|-3|3|-2|2
0|3|-3|1|-1|5
0|-5|-2|4|-1|3
1|4|-2|2|-1|1
1|-2|-2|0|-1|-5
|||-2|-4|0|6
|||-1|5|0|4
|||-1|3|0|2
|||-1|1|0|0
|||-1|-1|0|2
|||-1|-3|0|4
|||-1|-5|0|6
|||-1|-7|1|5
|||0|6|1|3
|||0|4|1|1
|||0|2|1|-3
|||0|0|1|-5
|||0|-2|2|4
|||0|-4
|||0|-6
|||1|5
|||1|3
|||1|1
|||1|-1
|||1|-3
|||1|-5
|||2|4
|||2|2
|||2|0
|||2|-2
|||3|1

In [5]:
import numpy as np

SCALE = 300
masslumps=np.array([[-1,4,1],
[0,3,1],
[0,-5,1],
[1,4,1],
[1,-2,1],
[-3,3,0],
[-3,1,0],
[-2,4,0],
[-2,2,0],
[-2,0,0],
[-2,-4,0],
[-1,5,0],
[-1,3,0],
[-1,1,0],
[-1,-1,0],
[-1,-3,0],
[-1,-5,0],
[-1,-7,0],
[0,6,0],
[0,4,0],
[0,2,0],
[0,0,0],
[0,-2,0],
[0,-4,0],
[0,-6,0],
[1,5,0],
[1,3,0],
[1,1,0],
[1,-1,0],
[1,-3,0],
[1,-5,0],
[2,4,0],
[2,2,0],
[2,0,0],
[2,-2,0],
[3,1,0],
[-2,2,-1],
[-1,5,-1],
[-1,3,-1],
[-1,1,-1],
[-1,-5,-1],
[0,6,-1],
[0,4,-1],
[0,2,-1],
[0,0,-1],
[0,2,-1],
[0,4,-1],
[0,6,-1],
[1,5,-1],
[1,3,-1],
[1,1,-1],
[1,-3,-1],
[1,-5,-1],
[2,4,-1]])

unit_com = np.mean(masslumps, axis=0)
center_of_mass = unit_com*SCALE
points_remap = masslumps*SCALE - center_of_mass
list(unit_com) # Center of mass coordinates in the form [x, y, z] in the unit cell

[-0.037037037037037035, 0.7962962962962963, -0.24074074074074073]

In [6]:
list(center_of_mass) # Center of mass coordinates in the form [x, y, z] in physical space

[-11.11111111111111, 238.88888888888889, -72.22222222222221]

To be sure that the new coordinate system is centered around the center of mass, we can calculate its center of mass and expect a point centered at the origin. The result will not be perfect due to computer float number precision.

In [10]:
list(np.mean(points_remap, axis=0)) # Should be [0, 0, 0]

[1.0947621410229691e-13, 6.736997790910579e-14, -3.0527021240063567e-14]

Now, knowing that the total mass of the asteroid is $5.10^{13}kg$, it is possible to deduce the mass of each lumps.

In [7]:
total_mass = 5e13 # Total mass of the system (in kg)
mass_per_point = total_mass / len(masslumps)
mass_per_point # Mass of each point (in kg)

925925925925.9259

Now, it is possible to calculate the inertia matrix as well as the three principal moments of inertia with the formula:

$$
I=
\left(
\begin{array}{ccc}
I_{xx} & I_{xy} & I_{xz} \\
I_{xy} & I_{yy} & I_{yz} \\
I_{xz} & I_{yz} & I_{zz} \\
\end{array}
\right)
=
\left(
\begin{array}{ccc}
\sum{m(y^2 + z^2)} & -\sum{mxy}         & -\sum{mxz} \\
-\sum{mxy}         & \sum{m(x^2 + z^2)} & -\sum{myz} \\
-\sum{mxz}         & -\sum{myz}         & \sum{m(x^2 + y^2)} \\
\end{array}
\right)
$$

For principal moments of inertia, it is the eigen values of this matrix.

In [8]:
def inertia_moment(mass, x, y):
    return mass * (x**2 + y**2)
def inertia_product(mass, x, y):
    return mass * x * y

inertia = np.array([[0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]], dtype=float)

m = mass_per_point
for [x, y, z] in points_remap:
    inertia += np.array([[inertia_moment(m, y, z), -inertia_product(m, x, y), -inertia_product(m, x, z)],
                         [-inertia_product(m, x, y), inertia_moment(m, x, z), -inertia_product(m, y, z)],
                         [-inertia_product(m, x, z), -inertia_product(m, y, z), inertia_moment(m, x, y)]])

inertia

array([[ 5.73858025e+19, -2.16049383e+17,  4.01234568e+16],
       [-2.16049383e+17,  9.31635802e+18,  1.72067901e+18],
       [ 4.01234568e+16,  1.72067901e+18,  6.33904321e+19]])

In [9]:
w,_=np.linalg.eig(inertia)
print('E-value:', w) # Eigenvalues of the inertia tensor (in kg m^2) in the form [Ixx, Iyy, Izz]

E-value: [9.26067960e+18 5.73866005e+19 6.34453125e+19]


## Spin kinetic energy

Knowing that the spin kinetic energy is given by: 
$$E=\sum_{k=1}^3{\frac{1}{2}I_k\Omega_k^2}$$
By differentiating the expression with respect to time:
$$\frac{dE}{dt}=\sum_{k=1}^3{I_k\Omega_k\frac{d\Omega_k}{dt}}$$
Or, according to Euler’s equations:
$$\frac{d\Omega_x}{dt}+\frac{(I_z-I_y)\Omega_y\Omega_z}{I_x}=\frac{Q_x}{I_x}$$
$$\frac{d\Omega_y}{dt}+\frac{(I_x-I_z)\Omega_z\Omega_x}{I_y}=\frac{Q_y}{I_y}$$
$$\frac{d\Omega_z}{dt}+\frac{(I_y-I_x)\Omega_x\Omega_y}{I_z}=\frac{Q_z}{I_z}$$
Without external torque $Q=0$.

$$\frac{d\Omega_x}{dt}=-\frac{(I_z-I_y)\Omega_y\Omega_z}{I_x}$$
$$\frac{d\Omega_y}{dt}=-\frac{(I_x-I_z)\Omega_z\Omega_x}{I_y}$$
$$\frac{d\Omega_z}{dt}=-\frac{(I_y-I_x)\Omega_x\Omega_y}{I_z}$$

Thus, combining this into the differentiation of the first equation:

$$\frac{dE}{dt}=\left(I_y-I_z+I_z-I_x+I_x-I_y\right)\prod_{k=1}^3\Omega_k$$

This means that $\frac{dE}{dt}=0$. Therefore, E is constant if any external torque is apply on the system.